In [108]:
import pandas as pd
import json
import googlemaps
import time
import folium

In [109]:
with open("../config/googleAPI.json") as f:
    config = json.load(f)
    GOOGLE_API_KEY = config["GOOGLE_API_KEY"]

In [110]:
import googlemaps
from datetime import datetime

# set up API key and client
gmaps = googlemaps.Client(key=GOOGLE_API_KEY)

# set up search query for "惠康" in Hong Kong
search_query = "惠康 Hong Kong"


In [111]:
def search_all():

    df = pd.DataFrame()

    # Lantau Island
    results = gmaps.places(
        query=search_query,
        location=(22.271728, 113.953225),
        radius=5000,
        )
    df = pd.concat([df, pd.DataFrame(results["results"])], axis=0)
    print("Total number of results: ", len(df))

    while "next_page_token" in results:
        time.sleep(2)
        results = gmaps.places(
            query=search_query,
            location=(22.271728, 113.953225),
            radius=5000,
            page_token=results["next_page_token"],
            )
        df = pd.concat([df, pd.DataFrame(results["results"])], axis=0)
        print("Total number of results: ", len(df))


    # Hong Kong Island
    results = gmaps.places(
        query=search_query,
        location=(22.267006, 114.185718),
        radius=5000,
        )
    df = pd.concat([df, pd.DataFrame(results["results"])], axis=0)
    print("Total number of results: ", len(df))

    while "next_page_token" in results:
        time.sleep(2)
        results = gmaps.places(
            query=search_query,
            location=(22.267006, 114.185718),
            radius=5000,
            page_token=results["next_page_token"],
            )
        df = pd.concat([df, pd.DataFrame(results["results"])], axis=0)
        print("Total number of results: ", len(df))
    
    # Tin Shui Wai
    results = gmaps.places(
        query=search_query,
        location=(22.420820, 113.988650),
        radius=5000,
        )
    df = pd.concat([df, pd.DataFrame(results["results"])], axis=0)
    print("Total number of results: ", len(df))

    while "next_page_token" in results:
        time.sleep(2)
        results = gmaps.places(
            query=search_query,
            location=(22.420820, 113.988650),
            radius=5000,
            page_token=results["next_page_token"],
            )
        df = pd.concat([df, pd.DataFrame(results["results"])], axis=0)
        print("Total number of results: ", len(df))

    # Tai Po
    results = gmaps.places(
        query=search_query,
        location=(22.434396, 114.190205),
        radius=5000,
        )
    df = pd.concat([df, pd.DataFrame(results["results"])], axis=0)
    print("Total number of results: ", len(df))

    while "next_page_token" in results:
        time.sleep(2)
        results = gmaps.places(
            query=search_query,
            location=(22.434396, 114.190205),
            radius=5000,
            page_token=results["next_page_token"],
            )
        df = pd.concat([df, pd.DataFrame(results["results"])], axis=0)
        print("Total number of results: ", len(df))

    # Kowloon
    results = gmaps.places(
        query=search_query,
        location=(22.331288, 114.204035),
        radius=5000,
        )
    df = pd.concat([df, pd.DataFrame(results["results"])], axis=0)
    print("Total number of results: ", len(df))

    while "next_page_token" in results:
        time.sleep(2)
        results = gmaps.places(
            query=search_query,
            location=(22.331288, 114.204035),
            radius=5000,
            page_token=results["next_page_token"],
            )
        df = pd.concat([df, pd.DataFrame(results["results"])], axis=0)
        print("Total number of results: ", len(df))
    
    # Tsuen Wan
    results = gmaps.places(
        query=search_query,
        location=(22.364287, 114.122800),
        radius=5000,
        )
    df = pd.concat([df, pd.DataFrame(results["results"])], axis=0)
    print("Total number of results: ", len(df))

    while "next_page_token" in results:
        time.sleep(2)
        results = gmaps.places(
            query=search_query,
            location=(22.364287, 114.122800),
            radius=5000,
            page_token=results["next_page_token"],
            )
        df = pd.concat([df, pd.DataFrame(results["results"])], axis=0)
        print("Total number of results: ", len(df))

    return df

In [112]:
df = search_all()

Total number of results:  20
Total number of results:  40
Total number of results:  60
Total number of results:  80
Total number of results:  100
Total number of results:  120
Total number of results:  140
Total number of results:  160
Total number of results:  180
Total number of results:  200
Total number of results:  220
Total number of results:  240
Total number of results:  260
Total number of results:  280
Total number of results:  300
Total number of results:  320
Total number of results:  340
Total number of results:  360


In [113]:
df.drop_duplicates(subset="place_id", inplace=True)
df_filtered = df[[ 'name', 'formatted_address','place_id', 'rating', 'user_ratings_total','types']]
df_filtered.loc[:,'latitude'] = df['geometry'].apply(lambda x: x['location']['lat'])
df_filtered.loc[:,'longitude'] = df['geometry'].apply(lambda x: x['location']['lng'])
df_filtered.reset_index(drop=True, inplace=True)
df_filtered

/var/folders/gz/rp9x0dzd5s13dyt0nv94f50h0000gn/T/ipykernel_5468/1490015204.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.loc[:,'latitude'] = df['geometry'].apply(lambda x: x['location']['lat'])
/var/folders/gz/rp9x0dzd5s13dyt0nv94f50h0000gn/T/ipykernel_5468/1490015204.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.loc[:,'longitude'] = df['geometry'].apply(lambda x: x['location']['lng'])


,name,formatted_address,place_id,rating,user_ratings_total,types,latitude,longitude
0,Wellcome,"84-90號 Queen's Road Central, Central, Hong Kong",ChIJ5b0oWHsABDQRJUwc57avQ94,3.8,362,"[supermarket, grocery_or_supermarket, store, f...",22.283245,114.155105
1,wellcome,"125, Metro Harbour View Plaza, 8 Fuk Lee St, T...",ChIJfzdi4rAABDQRYsiJViBAKIs,3.7,259,"[supermarket, grocery_or_supermarket, store, f...",22.324410,114.160454
2,Wellcome,"319 Nathan Rd, Jordan, Hong Kong",ChIJG3W4guwABDQRsYiKQEyeUvU,3.6,275,"[supermarket, grocery_or_supermarket, store, f...",22.306436,114.171362
3,Wellcome,"Shop No.23, G/F, Hoi Fu Shopping Centre, 2 Hoi...",ChIJvdn9MLkABDQRbEofjcY9gsc,3.6,215,"[supermarket, grocery_or_supermarket, store, f...",22.315173,114.164415
4,Wellcome,"G/F, TG Place, 8 Shing Yip St, Kwun Tong, Hong...",ChIJabudQ0UBBDQRF2ggpcEzLiE,3.5,182,"[supermarket, grocery_or_supermarket, store, f...",22.310417,114.225745
...,...,...,...,...,...,...,...,...
118,Wellcome Superstore,"69 Broadway, Mei Foo Sun Chuen, Hong Kong",ChIJT_ciK6v4AzQRs67ZP901Mik,3.8,147,"[supermarket, grocery_or_supermarket, store, f...",22.335935,114.140741
119,Wellcome,"Hong Kong, Tsing Yi, 長 青 村長 青 商業 中心 地下 T1-T5 號",ChIJl-Hd7M34AzQRxa6yYEDFlaY,3.1,24,"[supermarket, grocery_or_supermarket, store, f...",22.345011,114.109215
120,Wellcome,"G/F, No, 26-30 Wo Yi Hop Rd, Kwai Chung, Hong ...",ChIJ-5B442j5AzQR1WY8EluYXEI,3.5,24,"[supermarket, grocery_or_supermarket, store, f...",22.365888,114.136964
121,Wellcome Supermarket,"608號 Lai Chi Kok Rd, Cheung Sha Wan, Hong Kong",ChIJ11HrihcBBDQRdA9WzXizBeY,0.0,0,"[supermarket, grocery_or_supermarket, store, f...",22.333536,114.154778


In [114]:
df_filtered.to_csv("../data/processed/wellcomeMap.csv", index=False)

# Map Showing

In [115]:

# create a map centered at Hong Kong
m = folium.Map(location=[22.3193, 114.1694], zoom_start=11)

# iterate through the rows of the dataframe and add markers to the map
for index, row in df_filtered.iterrows():
    name = row['formatted_address']
    lat = row['latitude']
    lon = row['longitude']
    popup = folium.Popup(name)
    marker = folium.Marker(location=[lat, lon], popup=popup)
    marker.add_to(m)

# display the map
m


In [116]:
m.save("../data/processed/map.html")